In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [21]:
def parse_tree_adj(node, check_flag):
    res_list = []
    dep_counter = 0
    for c in node.children:
        dep_counter += 1


    xcomp_flag = False
    neg_flag = False
    adv_to_consider = []
    for c in node.children:  
        res = parse_tree_adj(c, check_flag)

        if node.pos_ == 'ADJ':
            if c.dep_ == 'xcomp':
                xcomp_flag = True
                start =  node.text
                for n in c.subtree:
                    start = start + ' ' + n.text
                res = [start]

            #special case, not sure why this happens
            elif (c.dep_ == 'poss' or c.dep_ == 'poss') and (c.text =='Her' or c.text == 'her'):
                check_flag = True
            elif (c.dep_ == 'nsubj' or c.dep_ == 'nsubj') and (c.text =='Her' or c.text == 'her'):
                check_flag = True
        
        if node.pos_ == 'NOUN':
            if c.dep_ == 'amod':
                for i in range(0, len(res), 1):
                    res[i] = res[i] + ' '+node.text
        

                
            
                   

        #negation logic- assumes that the negative statement directly follows the negative dependency      
        if neg_flag == True:
            for i in range(0, len(res), 1):
                res[i] = neg_res + ' ' + res[i]
            neg_flag = False

        if c.dep_ == 'neg':
            neg_flag = True
            neg_res = c.text
        
        #check flag logic
        if node.pos_ =='AUX' or node.pos_ =='VERB':
            if (c.dep_ == 'nsubj' or c.dep_ == 'nsubjpass') and (c.text =='she' or c.text == 'She'):
                check_flag = True
                for adverb in adv_to_consider:
                    res.append(adverb)
            elif (c.dep_ == 'nsubj' or c.dep_ == 'nsubjpass') and c.text !='she' and c.text != 'She':
                check_flag = False
            elif c.dep_ == 'attr' and check_flag == False:
                adv_to_consider.append(c.text)

        
                
        if c.dep_ != 'advmod' and c.dep_ != 'npadvmod':
            res_list += res


    #base cases
    if node.pos_ == 'ADJ' and check_flag == True and xcomp_flag == False:
        res_list += [node.text]

    return res_list


#for traits
def parse_tree_noun(node, check_flag):
    res_list = []
    dep_counter = 0
    for c in node.children:
        dep_counter += 1

    poss_flag = False
    amod_flag = False
    for c in node.children:  
        res = parse_tree_noun(c, check_flag)

        #check flag logic
        if node.pos_ =='NOUN':
            if (c.dep_ == 'poss' or c.dep_ == 'poss') and (c.text =='Her' or c.text == 'her'):
                check_flag = True
                poss_flag = True
            elif (c.dep_ == 'poss' or c.dep_ == 'poss') and c.text !='Her' and c.text != 'her':
                check_flag = False
            elif check_flag == True and c.dep_ == 'amod':
                amod_flag = True
                res = parse_tree_adj(c, True)
                

        res_list += res

    
    if amod_flag == False and check_flag == True and node.pos_ == 'NOUN':
        res_list.append(node.text)
        
    return res_list

In [22]:
with open('trait_test.txt') as f:
    lines = f.readlines()

for line in lines:
    print(line[:-1])
    doc = nlp(line[:-1])
    for i in range(0, len(doc), 1):
        if doc[i].dep_ == 'ROOT':
            print(parse_tree_noun(doc[i], False) + parse_tree_adj(doc[i], False))

I think my favorite part of her was how bubbly she was.
['bubbly']
Lana was a little bit shy at first but as I got to know her more, I found her super interesting, cool, and funny.
['funny', 'cool', 'interesting']
I think she really embodies the characteristics of our sorority through her drive and friendliness.
['friendliness', 'drive']
Despite her outgoing and kind nature, she is also quite introspective and thoughtful, and her politness was remarkable.
['kind', 'outgoing', 'politness', 'thoughtful', 'introspective']
Despite her outgoing nature and kind persona, she is also quite introspective and thoughtful.
['outgoing', 'thoughtful', 'introspective']
She was very outgoing and the conversation flowed super well, but she was not the brightest and I mean that in a really nice way because she was trying very hard.
['outgoing', 'not brightest']
She was in a leadership sorority in high school and you can tell she is just a social butterfly.
['high school', 'social butterfly']
She's alway